In [1]:
# Import necessary packages
import os
import numpy as np
import matplotlib.pyplot as plt

import rasterio as rio
import earthpy as et

import ee
import geemap

In [2]:
# Initiate interactive map
Map = geemap.Map(center=(39.791424, -104.809968), zoom=10)
Map.add_basemap('HYBRID')  # Add Google Map

In [3]:
# Define ROI

# Set ROI as 10 mile (16,093 meters) radius around the intersection of
# Pena Blvd and E-470 (about the SW corner of the airport)
roi = ee.Geometry.Point([-104.746984, 39.834241]).buffer(16093)

# Add ROI to map
Map.addLayer(roi, {}, 'ROI', opacity=0.5)

In [4]:
# Define function to unmix Landsat 5 ee.Image object and add a timestamp
def unmix_L5(image_L5):
    """Unmix each pixel with the given endmembers, by computing the pseudo-inverse
    and multiplying it through each pixel. Returns an image of doubles with the same
    number of bands as endmembers. (Description from GEE docs).

    Parameters
    ----------
    image_L5 : ee.Image
        Any ee.Image object from Landsat 5 (must include at least the first 7 bands).

    Returns
    ------
    unmixed_image : ee.Image
        A 4-band ee.Image object.  Three bands, ('urban', 'veg', and 'water') are the
        per-pixel fraction that the .unmix() method determined that correspond with
        the three given endmember values.  The final band, 'system:time_start' is
        the time metadata associated with that image.
    """
    # Select bands 1-7
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    image = image_L5.select(bands)

    # Endmembers taken from GEE unmix() example script found here:
    # https://developers.google.com/earth-engine/image_transforms
    urban = [88, 42, 48, 38, 86, 115, 59]  # band_0
    veg = [50, 21, 20, 35, 50, 110, 23]  # band_1
    water = [51, 20, 14, 9, 7, 116, 4]  # band_2
    
    # Run the GEE unmix() method with to get per-pixel fractions that are non-negative
    # and sum-to-one
    unmixed_image = image.unmix([urban, veg, water], True, True).rename([
        'urban', 'veg', 'water'])
    
    # Add time band and return image - time stamp is divided by large number to avoid 
    # extra high (or low) scale values.
    return unmixed_image.addBands(image.metadata('system:time_start').divide(1e13))

In [5]:
# Filter Landsat 5 ImageCollection and map the unmix function over the collection
collection = ee.ImageCollection('LANDSAT/LT05/C01/T1').filterDate(
    ee.Date('1995-01-01'), ee.Date('2011-12-31')).filterBounds(roi).map(unmix_L5)

In [6]:
# Reduce the collection with the linear fit reducer - independent variable is
# followed by the dependent variable
linearFit = collection.select(
    ['system:time_start', 'urban']).reduce(ee.Reducer.linearFit())

In [7]:
# Clip the linearFit to the roi
clipped_linearFit = linearFit.clip(roi)

# Display the results
Map.addLayer(clipped_linearFit,
             {'min': 0, 'max': [-0.9, 8e-5, 1], 'bands': ['scale', 'offset', 'scale']}, 'fit')

# Add legend to map
legend_keys = ['High Positive Urban Trend', 'Positive Urban Trend',
               'Nearly No Urban Trend', 'Negative Urban Trend']
legend_colors = ['#0000FF', '#01FFFF', '#00FF1F', '#FFFF02']

Map.add_legend(legend_keys=legend_keys,
               legend_colors=legend_colors, position='bottomleft')

Map

Map(center=[39.791424, -104.809968], controls=(WidgetControl(options=['position'], widget=HBox(children=(Toggl…

# Interpret the Results

In [8]:
out_dir = os.path.join(et.io.HOME, 'Downloads')
filename = os.path.join(out_dir, 'linearFit.tif')


# Exports two files - one file per band ('filename.bandname.tif')
geemap.ee_export_image(clipped_linearFit, filename=filename, scale=90, region=roi, file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to /Users/richardudell/Downloads


In [9]:
# Confirm that linearFit.scale.tif appears in your Downloads folder
scale_path = os.path.join(et.io.HOME, 'Downloads', 'linearFit.scale.tif')

# Import scale (slope values for each pixel in the ROI)
with rio.open(scale_path) as src:
    scale_tif = src.read()

In [10]:
# Calculate the total number of pixels in the dataset
total = scale_tif < np.inf

# Interpret the slope values as either trending positive or negative (above
# or below zero)
above = scale_tif < 0
below = scale_tif > 0

# Confirm that there are no values at zero
at = scale_tif == 0

print('Number of pixels above zero: ', above.sum(),
      '\nBelow zero: ', below.sum(), 
      '\nAt zero: ', at.sum())

Number of pixels above zero:  21039 
Below zero:  145896 
At zero:  0


In [11]:
# Calculate the total area of the ROI
area_per_pixel = clipped_linearFit.pixelArea()
total_area_roi = area_per_pixel.reduceRegion(ee.Reducer.sum(), roi, 30)
total_area_roi = total_area_roi.getInfo()['area']

In [13]:
# Calculate the percent of pixels trending towards urban
trending_urban_perc = round(above.sum()/total.sum(), 2)

print(trending_urban_perc, '% of the pixels in the ROI are trending towards urban.')
print('This is the same as: ', int(trending_urban_perc * total_area_roi), ' square meters, or', )
print(round((trending_urban_perc * total_area_roi)/2.59e6, 8), ' square miles.')

0.13 % of the pixels in the ROI are trending towards urban.
This is the same as:  104614003  square meters, or
40.39150716  square miles.
